In [1]:
!pip install -q crewai 'crewai[tools]' langchain_groq --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.9.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
transformers 4.50.3 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [25]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

In [26]:
import os
from google.colab import userdata
#groq_api_key = userdata.get('groq_api_key')
#os.environ["GROQ_API_KEY"] = groq_api_key

#hf_api_key = userdata.get('huggingfacehub')
#os.environ["HUGGINGFACEHUB_API_KEY"] = hf_api_key

serper_api_key = userdata.get('serper_api')
os.environ["SERPER_API_KEY"] = serper_api_key

gemini_api_key = userdata.get('gemini_api')
os.environ["GEMINI_API_KEY"] = gemini_api_key

In [27]:
from crewai import LLM
gem_llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)

In [28]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='/content/Mine_resume.pdf')
#semantic_search_resume = MDXSearchTool(mdx='/content/Mine_resume.md')

In [29]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=gem_llm
)

In [30]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=gem_llm
)

In [31]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=gem_llm
)

In [15]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=gem_llm
)

In [16]:
# Agent 5: Resume Scorer
resume_scorer = Agent(
    role="Resume Scoring Expert",
    goal="Evaluate the resume based on job requirements and provide a score out of 100",
    tools=[read_resume, scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Resume Scoring Expert, you analyze resumes against job requirements "
        "and assign a score based on relevance, skills match, and overall presentation. "
        "Your evaluations help candidates understand how well their resume aligns with the job."
    ),
    llm=gem_llm
)


In [17]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [18]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [19]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [20]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [21]:
# Task for Resume Scorer Agent: Evaluate Resume and Provide Score
resume_scoring_task = Task(
    description=(
        "Evaluate the tailored resume (from the Resume Strategist) against the job requirements "
        "(from the Researcher) and provide a score out of 100. "
        "Include feedback on strengths and areas for improvement."
    ),
    expected_output=(
        "A resume score out of 100 and detailed feedback on how well the resume matches "
        "the job requirements, along with suggestions for improvement."
    ),
    output_file="resume_score_feedback.md",
    context=[research_task, resume_strategy_task],
    agent=resume_scorer
)

In [22]:
# Update the Crew with the new agent and task
job_application_crew = Crew(
    agents=[researcher, profiler, resume_strategist, interview_preparer, resume_scorer],
    tasks=[research_task, profile_task, resume_strategy_task, interview_preparation_task, resume_scoring_task],
    verbose=True
)

In [33]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/view/4208580604/?alternateChannel=search&refId=ow%2BnQONSCJcGZ2Um6oxIZw%3D%3D&trackingId=hmwC7p5iIxstalKR5yDTpg%3D%3D',
    'github_url': 'https://github.com/HaRsH00000007',
    'personal_writeup': """I am a passionate AI and NLP enthusiast with a knack for creating advanced solutions. Currently, I am working on a multi-agent AI system to align resumes with job descriptions, streamlining the recruitment process. I have hands-on experience in generative AI, chatbot development, and utilizing LLMs like Llama and GPT for innovative applications. My technical expertise spans Python, Langchain, ChromaDB, and Streamlit, reflecting my commitment to building impactful AI-driven tools."""
}

In [34]:
result = job_application_crew.kickoff(inputs=job_application_inputs)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b1077ac5-4252-4c0c-b8a0-1263dd672d75                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 9cb50ce8-c6e4-4455-88c2-ad4e1f381344
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 9cb50ce8-c6e4-4455-88c2-ad4e1f381344
│      Status: Executing Task...
└── 📋 Task: c6f3f6e0-ffa4-48ec-98d0-cf3eb92a7c53
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 9cb50ce8-c6e4-4455-88c2-ad4e1f381344
│      Status: Executing Task...
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
└── 📋 Task: c6f3f6e0-ffa4-48ec-98d0-cf3eb92a7c53
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 9cb50ce8-c6e4-4455-88c2-ad4e1f381344
│      Status: Executing Task...
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
└── 📋 Task: c6f3f6e0-ffa4-48ec-98d0-cf3eb92a7c53
       Status: Executing Task...
    └── 🤖 Agent: Personal Profiler for Engineers
            Status: In Progress

...

🤖 Agent: Resume Scoring Expert
    Status: In Progress

🚀 Crew: crew
├── 📋 Task: 9cb50ce8-c6e4-4455-88c2-ad4e1f381344
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
├── 📋 Task: c6f3f6e0-ffa4-48ec-98d0-cf3eb92a7c53
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: ✅ Completed
├── 📋 Task: d28ad693-b799-4d9a-ae20-96e0642b5788
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
├── 📋 Task: 18baa4fe-a9e4-4cdf-b208-59058a23ce1f
│      Assigned to: Engineering Interview Preparer
│      Status: ✅ Completed
│   └── 🤖 Agent: Engineering Interview Preparer
│           Status: ✅ Completed
└── 📋 Task: 07241acc-8a4b-412e-8427-be1d5aa6d6b7
       Status: Executing Task...
    └── 🤖 Agent: Resume Scoring Expert
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 9cb50ce8-c6e4-4455-88c2-ad4e1f381344
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
├── 📋 Task: c6f3f6e0-ffa4-48ec-98d0-cf3eb92a7c53
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: ✅ Completed
├── 📋 Task: d28ad693-b799-4d9a-ae20-96e0642b5788
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
├── 📋 Task: 18baa4fe-a9e4-4cdf-b208-59058a23ce1f
│      Assigned to: Engineering Interview Preparer
│      Status: ✅ Completed
│   └── 🤖 Agent: Engineering Interview Preparer
│           Status: ✅ Completed
└── 📋 Task: 07241acc-8a4b-412e-8427-be1d5aa6d6b7
       Assigned to: Resume Scoring Expert
       Status: ✅ Completed
    └── 🤖 Agent: Resume Scoring Expert
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 07241acc-8a4b-412e-8427-be1d5aa6d6b7                                                                     │
│  Agent: Resume Scoring Expert                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b1077ac5-4252-4c0c-b8a0-1263dd672d75                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯